In [ ]:
import pandas as pd
import os
from pathlib import Path
import sys
import pandas as pd
from datetime import datetime
import numpy as np
import csv

In [ ]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
else:
    data_path = Path(project_path, 'dataset')

utils_path = str(Path(project_path, 'utils'))
# including the project folder and the utils folder
if utils_path not in ''.join(sys.path):
    sys.path.extend([str(project_path), utils_path])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('')
print('sys.path = {0}'.format(sys.path))

In [ ]:
data_path = Path(data_path, 'clpsych19_test_data')
os.listdir(data_path)

In [ ]:
# read in shared_task_posts csv file
# filename = 'clpsych19_public_shared_task_posts.csv'
filename = 'shared_task_posts_test.csv'
# shared_post = pd.read_csv(Path(data_path, filename), header=None, 
#                                 names=['post_id', 'user_id', 'timestamp', 
#                                        'subreddits', 'post_title', 'post_body'])
shared_post = pd.read_csv(Path(data_path, filename))
print(shared_post.shape)
shared_post.head()

In [ ]:
filename = 'post_user_risk.csv'
users = pd.read_csv(Path(data_path, filename))
print(users.shape)
users.head()

In [ ]:
# read in task_B_train csv file
filename = 'risk_tbs.csv'
sentiments = pd.read_csv(Path(data_path, filename), header=None, 
                         names=['post_id', 'sent_1', 'sent_2', 'sent_3', 'sent_4', 'sent_5'])
# sentiments = sentiments.set_index('post_id').copy()
sentiments.post_id = sentiments.post_id.astype(int)
print(sentiments.shape)
sentiments.head()

In [ ]:
# read in task_B_train csv file
# filename = 'risk_tbs_num.csv'
filename = 'clpsych19_test_title_body_sentiments_sentence.csv'
# sentence_number = pd.read_csv(Path(data_path, filename),header=None, 
#                          names=['post_id', 'num_sentences'])
sentence_number = pd.read_csv(Path(data_path, filename),header=None, 
                         names=['user_id', 'post_id', 'num_sentences'])
print(sentence_number.shape)
sentence_number.head()

In [ ]:
# n_dataset = sentiments.copy()
# n_dataset['user_id'] = 0
# n_dataset['risk_label'] = 0
# n_dataset['post_id'] = 0
# for sent_ele_ix, sent_ele in sentiments.iterrows():
#     current_user = users.loc[int(sent_ele['post_id'])]
#     user_id = current_user['user_id']
#     n_dataset.loc[sent_ele_ix, 'user_id'] = user_id
#     n_dataset.loc[sent_ele_ix, 'risk_label'] = current_user['risk_label']
#     n_dataset.loc[sent_ele_ix, 'post_id'] = current_user['post_id']
# n_dataset.head()

In [ ]:
sntiments_num = pd.merge(sentiments, sentence_number, on='post_id', how='inner')
sntiments_num.head()
dataset = pd.merge(users, sntiments_num, left_index=True, right_on='post_id', how='inner')
dataset.head()

In [ ]:
dataset = dataset[['post_id_x','sent_1','sent_2','sent_3','sent_4','sent_5','user_id','risk_label', 'num_sentences']]
dataset.columns = ['post_id','sent_1','sent_2','sent_3','sent_4','sent_5','user_id','risk_label', 'num_sentences']
dataset.head()

In [ ]:
dataset.keys()

In [ ]:
dataset = pd.merge(dataset, shared_post, on=['post_id', 'user_id'], how='inner')
dataset = dataset[['post_id', 'sent_1', 'sent_2', 'sent_3', 'sent_4', 'sent_5', 'user_id',
       'risk_label', 'num_sentences', 'timestamp']].copy()
dataset.head()

In [ ]:
print(dataset.shape)
dataset = dataset[~pd.isnull(dataset['risk_label'])]  
print(dataset.shape)

In [ ]:
dataset.to_csv(Path(data_path, 'dataset_sentiments.csv'), index=False)

In [ ]:
filename = 'clpsych19_test_title_body_sentiments.csv'
dataset = pd.read_csv(Path(data_path, filename))
dataset.drop('correct', axis=1, inplace=True)
dataset.head()

In [ ]:
dataset.columns = ['user_id', 'post_id', 'sent_1', 'sent_2', 'sent_3', 'sent_4',
       'sent_5']

In [ ]:
n_dataset = pd.merge(dataset, shared_post, on=['post_id', 'user_id'], how='inner')
n_dataset.head()

In [ ]:
n_dataset.drop(['subreddit','post_title','post_body'], axis=1, inplace=True)

In [ ]:
n_dataset.head()

In [ ]:
n_dataset = pd.merge(n_dataset, sentence_number, on=['post_id', 'user_id'], how='inner')
n_dataset.head()

In [ ]:
n_dataset.to_csv(Path(data_path, 'dataset_sentiments_test.csv'), index=False)

In [ ]:
dataset = pd.read_csv(Path(data_path, 'dataset_sentiments_test.csv'))
dataset.head()

Individual post-level sentiment vector: sum of all sentiment vectors for sentences in a post / total number of the sentence in the post [you already computed it for each post]

In [ ]:
averaged_sentiments_list_per_post = list()

#iterate over user ids
for row_ix, row in dataset.iterrows():
    new_row = list()
    # individual post-level sentiment vector
    relevant_dist = row[['sent_1','sent_2','sent_3','sent_4','sent_5']] / row['num_sentences']
    # normalization
    sentiment_dist = relevant_dist / relevant_dist.sum()
    # store
#     new_row.extend(row[['post_id', 'user_id', 'risk_label','timestamp']])
    new_row.extend(row[['post_id', 'user_id','timestamp']])
    new_row.extend(list(sentiment_dist.values))
    averaged_sentiments_list_per_post.append(np.array(new_row))

In [ ]:
file_obj = Path(data_path, 'sentiment_per_post_test.csv')
with file_obj.open('w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
#     writer.writerow(['post_id', 'user_id', 'risk_label', 'timestamp', 'sent_1','sent_2',
#                      'sent_3','sent_4','sent_5'])
    writer.writerow(['post_id', 'user_id', 'timestamp', 'sent_1','sent_2',
                     'sent_3','sent_4','sent_5'])
    for row in averaged_sentiments_list_per_post:
        writer.writerow(row)

In [ ]:
individual_post_sent = pd.read_csv(file_obj)
individual_post_sent.head()

- All sentence-level average (micro ave.): sum of all sentiment vectors (1x5) across all the posts of a user / total number of sentences across all the posts of the user (Ni,s)

In [ ]:
averaged_user_sentiments_list_micro = list()

#iterate over user ids
for user in dataset.user_id.unique():
    user_posts = dataset[dataset['user_id'] == user]
    user_sentiments_list_micro = list()
    # get posts for current user
    total_sentences = user_posts['num_sentences'].sum()
    # All sentence-level average (micro ave.)
    relevant_dist_micro = user_posts[['sent_1','sent_2','sent_3','sent_4','sent_5']].sum() / total_sentences
    # normalization
    sentiment_dist_micro = relevant_dist_micro / relevant_dist_micro.sum()
    # store
    user_sentiments_list_micro.append(user)
    user_sentiments_list_micro.extend(list(sentiment_dist_micro.values))
#     user_sentiments_list_micro.append(user_posts['risk_label'].unique()[0])
    averaged_user_sentiments_list_micro.append(np.array(user_sentiments_list_micro))

In [ ]:
file_obj = Path(data_path, 'sentiment_per_user_micro_test.csv')
with file_obj.open('w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
#     writer.writerow(['user_id', 'sent_1_micro','sent_2_micro','sent_3_micro','sent_4_micro','sent_5_micro', 
#                      'risk_label'])
    writer.writerow(['user_id', 'sent_1_micro','sent_2_micro','sent_3_micro','sent_4_micro','sent_5_micro'])
    for row in averaged_user_sentiments_list_micro:
        writer.writerow(row)

In [ ]:
file_obj = Path(data_path, 'sentiment_per_user_micro_test.csv')
micro = pd.read_csv(file_obj).head()
micro.head()

- All post-level average (marco ave.): sum of Individual post-level sentiment vector for all the posts of a user / total number of posts of a user (Ni,p)

In [ ]:
averaged_user_sentiments_list_macro = list()

#iterate over user ids
for user in dataset.user_id.unique():
    user_sentiments_list_macro = list()
    user_posts = dataset[dataset['user_id'] == user]
    #get posts for current user
    total_post = user_posts.shape[0]
    # All sentence-level average (macro ave.)
    relevant_dist_macro = user_posts[['sent_1','sent_2','sent_3','sent_4','sent_5']].head() / total_post
    # normalization
    dist_sum_macro = relevant_dist_macro.sum()
    sentiment_dist_macro = dist_sum_macro / dist_sum_macro.sum()
    # store
    user_sentiments_list_macro.append(user)
    user_sentiments_list_macro.extend(list(sentiment_dist_macro.values))
#     user_sentiments_list_macro.append(user_posts['risk_label'].unique()[0])
    averaged_user_sentiments_list_macro.append(np.array(user_sentiments_list_macro))

In [ ]:
file_obj = Path(data_path, 'sentiment_per_user_macro_test.csv')
with file_obj.open('w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
#     writer.writerow(['user_id', 'sent_1_macro','sent_2_macro','sent_3_macro','sent_4_macro','sent_5_macro', 
#                      'risk_label'])
    writer.writerow(['user_id', 'sent_1_macro','sent_2_macro','sent_3_macro','sent_4_macro','sent_5_macro'])
    for row in averaged_user_sentiments_list_macro:
        writer.writerow(row)

In [ ]:
macro = pd.read_csv(Path(data_path, 'sentiment_per_user_macro_test.csv'))
macro.head()